In [1]:
import pandas as pd
import numpy as np
import geojson as gj

In [2]:
file_to_load = "../json_files/LA_Certificated_Dispensary_Locations.csv"
disp_df = pd.read_csv(file_to_load)

In [3]:
# dispURL = 'https://data.lacity.org/resource/bunu-zsyc.geojson'
# dispDF = pd.read_json(dispURL)

# dispDF.head()

In [4]:
disp_df.reset_index(drop = True, inplace = True)

In [5]:
disp_df[["lat","long"]] = disp_df.LOCATION.str.split(",",expand=True)

In [6]:
# disp_df.head()

In [7]:
# ********************DONT RUN MORE THAN ONCE OR YOU WILL NEED TO RESTART KERNAL
disp_df["lat"] = disp_df.lat.str.slice(start=1)

In [8]:
# ********************DONT RUN MORE THAN ONCE OR YOU WILL NEED TO RESTART KERNAL
disp_df["long"] = disp_df.long.str.slice(stop=-1)

In [9]:
disp_df["lat"] = pd.to_numeric(disp_df["lat"])
disp_df["long"] = pd.to_numeric(disp_df["long"])
# disp_df.head()

In [10]:
# disp_df.to_json(r'../JSON Files/DISP.JSON', orient = "index")
disp_df.dtypes

LOCATION ACCOUNT #       object
BUSINESS NAME            object
DBA NAME                 object
CONFIRMED                object
STREET ADDRESS           object
CITY                     object
ZIP CODE                 object
LOCATION DESCRIPTION     object
MAILING ADDRESS          object
MAILING CITY             object
MAILING ZIP CODE         object
COUNCIL DISTRICT          int64
LOCATION START DATE      object
LOCATION                 object
lat                     float64
long                    float64
dtype: object

In [13]:
def data2geojson(df, filename):
    features = []
    insert_features = lambda X: features.append(
            gj.Feature(geometry=gj.Point((X["long"],
                                                    X["lat"]
#                                           ,X["elev"]
                                         )),
                            properties=dict(zip_code=X["ZIP CODE"],
                                            business_name = X["BUSINESS NAME"],
                                            street_address = X["STREET ADDRESS"],
                                           location_start_date = X["LOCATION START DATE"])))
    df.apply(insert_features, axis=1)
    with open(filename, 'w', encoding='utf8') as fp:
        gj.dump(gj.FeatureCollection(features), fp, sort_keys=True, ensure_ascii=False)

In [14]:
outputFilePath = '../static/data/'
dispFileName = f'{outputFilePath}disp.geojson'
data2geojson(disp_df, dispFileName)